In [1]:
import os
os.chdir("/home/ec2-user/SageMaker/foodi-ml/")

In [2]:
#!pip install -r requirements.txt

In [3]:
import os
import torch
from tqdm import tqdm

import params
from retrieval.train import train
from retrieval.utils import helper
from retrieval.model import loss
from retrieval.model.model import Retrieval
from retrieval.data.loaders import get_loaders
from retrieval.utils.logger import create_logger
from retrieval.utils.helper import load_model
from retrieval.utils.file_utils import load_yaml_opts, parse_loader_name

In [4]:
from addict import Dict

# Functions

In [5]:
def get_data_path(opt):
    if 'DATA_PATH' not in os.environ:
        if not opt.dataset.data_path:
            raise Exception('''
                DATA_PATH not specified.
                Please, run "$ export DATA_PATH=/path/to/dataset"
                or add path to yaml file
            ''')
        return opt.dataset.data_path
    else:
        return os.environ['DATA_PATH']

In [27]:
def get_tokenizers(train_loader):
    tokenizers = train_loader.dataset.tokenizer
    if type(tokenizers) != list:
        tokenizers = [tokenizers]
    return tokenizers

# Run

In [6]:
os.environ["DATA_PATH"] = "/home/ec2-user/SageMaker/data/"

In [7]:
options = "options/adapt/foodi-ml/i2t.yaml"

In [8]:
args = {
    "options": options,
}
args = Dict(args)
opt = load_yaml_opts(args.options)

In [9]:
logger = create_logger(level='debug' if opt.engine.debug else 'info')
#logger.info(f'Used args   : \n{args}')
#logger.info(f'Used options: \n{opt}')

In [10]:
# Get path of the data
data_path = get_data_path(opt)

In [11]:
# Get loaders
train_loader, val_loaders, adapt_loaders = get_loaders(data_path, args.local_rank, opt)

2021-08-16 14:57:25,680 - [INFO    ] - Loaded vocab containing 2487 tokens
2021-08-16 14:57:25,681 - [INFO    ] - Loaded from .vocab_cache/foodiml_vocab.json.
2021-08-16 14:57:25,681 - [INFO    ] - Created tokenizer with init 2487 tokens.
2021-08-16 14:57:25,726 - [INFO    ] - [FoodiML] Loaded 8011 images and 8011 annotations.
2021-08-16 14:57:25,731 - [INFO    ] - Loaded vocab containing 2487 tokens
2021-08-16 14:57:25,732 - [INFO    ] - Loaded from .vocab_cache/foodiml_vocab.json.
2021-08-16 14:57:25,732 - [INFO    ] - Created tokenizer with init 2487 tokens.
2021-08-16 14:57:25,769 - [INFO    ] - [FoodiML] Loaded 0 images and 0 annotations.
2021-08-16 14:57:25,770 - [INFO    ] - Adapt loaders: 0


In [28]:
tokenizers = get_tokenizers(train_loader)

In [29]:
tokenizers